In [1]:
import pandas as pd
import numpy as np

# 制造30w假数据
def data_generator(count):
    count = count
    df1 = np.random.randn(count)
    df2 = np.random.randn(count)
    df3 = np.random.randn(count)
    df4 = np.random.randn(count)
    df5 = np.random.randn(count)

    codes = np.random.randint(0,4800,size=count)
    dates = pd.date_range('20090101', '20171030',freq='5Min') 

    mydict = {'code':codes,'date':dates[:count],'a':df1,'b':df2,'c':df3,'d':df4,'e':df5}
    df = pd.DataFrame(mydict)
    return df

In [2]:
import pymongo
import time


# 连接数据库
client = pymongo.MongoClient('127.0.0.1', 27017)
dblist = client.list_database_names()
print(dblist)
db = client.test
col = db['insert_test']

def clear_data():
    print('开始清理数据库')
    for x in col.find():
        col.delete_one(x)
    print('数据库清理完成')

['admin', 'config', 'local', 'quant', 'test']


In [3]:
# 一次性插入30w数据
import math
def upsert_once(data_frame,slice):
    
    num = math.ceil(data_frame.shape[0]/slice)
    for j in range(num):
        operations = []
        to_insert_df = data_frame[j*slice:(j+1)*slice]
        for i in range(to_insert_df.shape[0]):
            operations.append(
                pymongo.UpdateOne(
                    {"code": str(data_frame.iloc[i].code),"date": data_frame.iloc[i].date},
                    {
                        "$set": {
                            "a": float(data_frame.iloc[i].a),
                            "b": float(data_frame.iloc[i].b),
                            "c": float(data_frame.iloc[i].c),
                            "d": float(data_frame.iloc[i].d),
                            "e": float(data_frame.iloc[i].e)
                        }
                    },
                    upsert=True,
                )
            )
            print(f'批量插入操作 {i+1}/{to_insert_df.shape[0]}||{j+1}/{num}',end="\r")
        rs = col.bulk_write(operations)
        print('批量插入完成',end="\r")
    #     print(rs)
        col.create_index([("code", 1), ("date", 1)])
        print('索引建立完成',end="\r")
print('方法注册')

方法注册


In [8]:
clear_data()


df = data_generator(200000)



# 插入操作
print(df.shape[0])
loop_count=10
slice_num = 200000
elapse=0
for i in range(loop_count):
    print(f'重复执行{i+1}/{loop_count}',end="\r")
    start_time = time.time()
    upsert_once(data_frame=df,slice=slice_num)
    end_time = time.time()
    elapse+=end_time-start_time
# 插入操作end

print(f'重复{loop_count}次，平均每条数据插入时间为{elapse/loop_count}')


开始清理数据库
数据库清理完成
200000
重复10次，平均每条数据插入时间为267.63216943740844


## 模拟20w数据插入

| 数据库空 | 单次量 | 线程 | 循环次数 | 平均耗时 |
|  ----  | ----  |  ----  | ----  |  ----  |
|  YES   | 200  |  1   | 10  |  254.72016043663024   |
|  YES   | 2000  |  1   | 10  |  268.07236483097074   |
|  YES   | 20000  |  1   | 10  |  257.3907039403915   |
|  YES   | 200000  |  1   | 10  |  257.3907039403915   |